# Image Classification of Documents

## 1. Setup
To prepare your environment, you need to install some packages and enter credentials for the Watson services.

### 1.1 Install the necessary packages
You need the latest versions of these packages:
python-swiftclient: is a python client for the Swift API.

### Install IBM Cloud Object Storage Client: 

In [1]:
!pip install ibm-cos-sdk

Requirement not upgraded as not directly required: ibm-cos-sdk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: ibm-cos-sdk-s3transfer==2.*,>=2.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk)
Requirement not upgraded as not directly required: ibm-cos-sdk-core==2.*,>=2.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk)
Requirement not upgraded as not directly required: python-dateutil<3.0.0,>=2.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk)
Requirement not upgraded as not directly required: docutils>=0.10 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk)
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk)
Re

### Now restart the kernel by choosing Kernel > Restart.

### 1.2 Import packages and libraries
Import the packages and libraries that you'll use:

In [2]:
import os, random
import numpy as np
import pandas as pd
import PIL
import keras
import itertools
from PIL import Image
import ibm_boto3
from botocore.client import Config


from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


Using TensorFlow backend.
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 2. Configuration
Add configurable items of the notebook below

### 2.1 Add your service credentials for Object Storage
You must create Object Storage service on IBM Cloud. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Insert the Object Storage Streaming Body credentials and ensure the variable is referred as  streaming_body_1 in the following cell after removing the current contents in the cell.

In [3]:

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_6320bca5848940229cbd3b964d36bb3e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='54gCzvBIVoqsU7ChZzFrNF_rYb6Y8e0n3n4NLCqX5fr_',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_6320bca5848940229cbd3b964d36bb3e.get_object(Bucket='smruthic94609ce4ae94509bc95665597fb0968', Key='Data_Upload.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 



### 2.2 Global Variables 
Enter the batch size for training, testing and validation dataset

In [4]:
batch_size_train = 20
batch_size_val = 10
batch_size_test = 25

# 3. Storage

## 3.1 Extract the Dataset 

Input the zip file from object storage and extract the data onto the /home/dsxuser/work folder

In [5]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
paths = zip_ref.namelist()
for path in paths:
    print(zip_ref.extract(path))
zip_ref.close()

/home/dsxuser/work/Data_Upload
/home/dsxuser/work/Data_Upload/test
/home/dsxuser/work/Data_Upload/test/Cheque
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_4.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_5.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_6.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_7.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_8.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_26.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_27.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_28.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_29.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_30.jpg
/home/dsxuser/work/Data_Upload/test/None
/home/dsxuser/work/Data_Upload/test/None/none_15.jpg
/home/dsxuser/work/Data_Upload/test/None/none_16.jpg
/home/dsxuser/work/Data_Upload/test/None/

# 4. Classification

## 4.1 Create the Datset

In [6]:
'''Converting Data Format according to the backend used by Keras
'''
keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

In [7]:
'''Input the Training Data
'''
train_path = '/home/dsxuser/work/Data_Upload/Trial_train/'
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['Cheque', 'Driving_License','None','Pancard','Passport'], batch_size=batch_size_train)
type(train_batches)

Found 191 images belonging to 5 classes.


keras.preprocessing.image.DirectoryIterator

In [8]:
'''Input the Validation Data
'''

val_path = '/home/dsxuser/work/Data_Upload/Trial_val/'
val_batches = ImageDataGenerator().flow_from_directory(val_path, target_size=(224,224), classes=['Cheque', 'Driving_License','None','Pancard','Passport'], batch_size=batch_size_val)


Found 50 images belonging to 5 classes.


In [9]:
'''Input the Test Data
'''
test_path = '/home/dsxuser/work/Data_Upload/test/'
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['Cheque', 'Driving_License','None','Pancard','Passport'], batch_size=batch_size_test)


Found 25 images belonging to 5 classes.


In [10]:
test_imgs, test_labels = next(test_batches)
test_labels

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]], dtype=float32)

In [11]:
y_test= [ np.where(r==1)[0][0] for r in test_labels ]
y_test

[3, 0, 0, 4, 4, 1, 1, 3, 3, 1, 1, 0, 4, 4, 0, 0, 3, 1, 2, 4, 2, 3, 2, 2, 2]

## 4.2 Build the Model

In [12]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary() # The output layer gives 1000, this model has 1000 classes

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
type(vgg16_model) #This is a Keras Functional API need to convert to sequential
model = Sequential() #Iterate over the functional layers and add it as a stack
for layer in vgg16_model.layers:
    model.add(layer)

In [14]:
model.layers.pop() # We dont want 1000 classes, so remove the last layer
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
for layer in model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

In [16]:
model.add(Dense(5, activation='sigmoid')) # Add the last layer
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [17]:
# Complie the model
model.compile(Adam(lr=.00015), loss='categorical_crossentropy', metrics=['accuracy'])

## 4.3 Train the Model

In [18]:
model.fit_generator(train_batches, steps_per_epoch=20, 
                    validation_data=val_batches, validation_steps=20, epochs=5, verbose=1)

Epoch 1/5
20/20 [==============================] - 647s 32s/step - loss: 1.6024 - acc: 0.4574 - val_loss: 1.5968 - val_acc: 0.7200
Epoch 2/5
20/20 [==============================] - 646s 32s/step - loss: 1.5902 - acc: 0.7890 - val_loss: 1.5882 - val_acc: 0.8400
Epoch 3/5
20/20 [==============================] - 647s 32s/step - loss: 1.5803 - acc: 0.9248 - val_loss: 1.5804 - val_acc: 0.9200
Epoch 4/5
20/20 [==============================] - 646s 32s/step - loss: 1.5732 - acc: 0.9724 - val_loss: 1.5758 - val_acc: 0.8600
Epoch 5/5
20/20 [==============================] - 646s 32s/step - loss: 1.5667 - acc: 0.9825 - val_loss: 1.5714 - val_acc: 0.9000


## 4.4 Test the Model with External Test Images

In [19]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)
predictions

array([[ 0.49045312,  0.48321047,  0.48106536,  0.51799881,  0.48700282],
       [ 0.51645035,  0.48334283,  0.47801626,  0.50063127,  0.50346553],
       [ 0.51644331,  0.48346132,  0.47810188,  0.50060511,  0.50322056],
       [ 0.47638094,  0.5004338 ,  0.49348253,  0.48333481,  0.51741409],
       [ 0.50119907,  0.48831484,  0.5204407 ,  0.4805955 ,  0.48646328],
       [ 0.47953597,  0.51964068,  0.50977141,  0.47993723,  0.48288852],
       [ 0.47959635,  0.51952171,  0.50971037,  0.47995955,  0.48302555],
       [ 0.49047604,  0.48316553,  0.48100588,  0.51805109,  0.48692498],
       [ 0.48975655,  0.48563409,  0.48283672,  0.51567882,  0.48692626],
       [ 0.4795393 ,  0.51961815,  0.50976795,  0.47991693,  0.4829081 ],
       [ 0.49668938,  0.51255488,  0.49431124,  0.49384063,  0.49430156],
       [ 0.51634961,  0.48349392,  0.47844103,  0.50067508,  0.50323087],
       [ 0.47662729,  0.50577444,  0.50625366,  0.49245915,  0.52240807],
       [ 0.476302  ,  0.50052953,  0.4

In [26]:
predictions
y_pred=[]
for i in range(len(predictions)):
    y_pred.append(np.argmax(predictions[i]))
print(y_pred)
#plots(test_imgs, titles=y_pred)

ctr=0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        ctr=ctr+1
res = ctr/len(y_pred)*100
print(res)


[3, 0, 0, 4, 2, 1, 1, 3, 3, 1, 1, 0, 4, 4, 0, 0, 3, 1, 2, 4, 2, 3, 4, 2, 2]
23
92.0
